In [47]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os,sys
import plyfile
import numpy as np
import argparse
import h5py
import liblas


In [77]:
args = argparse.Namespace(
#     datafolder = '../data/Miami-Dade/LAS',
    datafolder = '../data/Amsterdam/Utrecht-test-2'
)

In [54]:
import time
start_time_dict = {}
total_time_dict = {}

current_time_ms = lambda:int(round(time.time()*1000))

def timer_start(msg): 
    global start_time_dict
    global total_time_dict
    start_time_dict[msg] = current_time_ms()
    if not msg in total_time_dict:
        total_time_dict[msg] = 0
    
def timer_pause(msg):
    global start_time_dict
    global total_time_dict
    total_time_dict[msg] += current_time_ms() - start_time_dict[msg]
    
def timer_stop(msg):
    global total_time_dict
    timer_pause(msg)
    print("{} completed in {}ms".format(msg, total_time_dict[msg]))
    total_time_dict[msg] = 0

In [55]:
def read_xyz_label_from_las(filename_las):
    msg = 'Loading {}...'.format(filename_las)
    timer_start(msg)
    f = liblas.file.File(filename_las, mode='r')    
    h = f.header
    xyzirgb_num = h.point_records_count
    xyz = np.ndarray((xyzirgb_num, 3))
    labels = np.ndarray(xyzirgb_num, np.int16)
    i = 0
    for p in f:
        xyz[i] = [p.x, p.y, p.z]
        labels[i] = p.classification
        i += 1
    timer_stop(msg)
    return xyz, labels, xyzirgb_num

def save_xyz_label_to_las(filename_las, xyz, labels):  
    msg = 'Saving {}...'.format(filename_las)
    timer_start(msg)
    h = liblas.header.Header()
    h.dataformat_id = 1
    h.major = 1
    h.minor = 2
    h.min = np.min(xyz, axis=0)
    h.max = np.max(xyz, axis=0)
    h.scale = [1e-3, 1e-3, 1e-3]
    
    f = liblas.file.File(filename_las, mode='w', header=h)    
    for i in range(xyz.shape[0]):
        p = liblas.point.Point()
        p.x = xyz[i,0] / h.scale[0]
        p.y = xyz[i,1] / h.scale[1]
        p.z = xyz[i,2] / h.scale[2]
        p.classification = labels[i]
        p.color = liblas.color.Color()
        p.intensity = 100
        p.return_number = 1
        p.number_of_returns = 1
        p.scan_direction = 1
        p.scan_angle = 0
        f.write(p)
#         if i > 10000:
#             break
    f.close()
    timer_stop(msg)


In [51]:
def get_pred_prefixes(datafolder):
    fs = os.listdir(datafolder)
    preds = []
    for f in fs:
        if f[-8:] == '_pred.h5':
            preds += [f]
    pred_pfx = []
    for p in preds:
        if '_zero' in p:
            pred_pfx += [p.split('_zero')[0]]
    return np.unique(pred_pfx)


In [ ]:
# parser = argparse.ArgumentParser()
# parser.add_argument('--datafolder', '-d', help='Path to input *_pred.h5', required=True)
# parser.add_argument('--version', '-v', help='full or reduced', type=str, required=True)
# args = parser.parse_args()
# print(args)

SAVE_TO_EXT = '.las'
LOAD_FROM_EXT = '.las'
# SAVE_TO_EXT = '.txt'
# LOAD_FROM_EXT = '.txt'


categories_list = get_pred_prefixes(args.datafolder)
# categories_list = ['amsterdam_las_' + ('0000' + str(category))[-5:] for category in length_dict]
# categories_list = [category for category in length_dict]
print(categories_list)

for category in categories_list:
    output_path = os.path.join(args.datafolder,"results",category + "_pred" + SAVE_TO_EXT)
    if not os.path.exists(os.path.join(args.datafolder,"results")):
        os.makedirs(os.path.join(args.datafolder,"results"))
    pred_list = [pred for pred in os.listdir(args.datafolder)
                 if category in pred  and pred.split(".")[0].split("_")[-1] == 'pred']
    print('pred_list: {}'.format(pred_list))

#     label_length = length_dict[category][0]
#     merged_label = np.zeros((label_length),dtype=int)
#     merged_confidence = np.zeros((label_length),dtype=float)
    merged_label = None
    merged_confidence = None

    for pred_file in pred_list:
        print(os.path.join(args.datafolder, pred_file))
        data = h5py.File(os.path.join(args.datafolder, pred_file))
        labels_seg = data['label_seg'][...].astype(np.int64)
        indices = data['indices_split_to_full'][...].astype(np.int64)
        confidence = data['confidence'][...].astype(np.float32)
        data_num = data['data_num'][...].astype(np.int64)
        
#         print('labels_seg: {}'.format(labels_seg.shape))
#         print('indices: {}'.format(indices.shape))
#         print('confidence: {}'.format(confidence.shape))
#         print('data_num: {}'.format(data_num.shape))

        if merged_label is None:
            #calculating how many labels need to be there in the output
            label_length = 0
            for i in range(indices.shape[0]):
                label_length = np.max([label_length, np.max(indices[i][:data_num[i]])])
            label_length += 1
            merged_label = np.zeros((label_length),dtype=int)
            merged_confidence = np.zeros((label_length),dtype=float)
        else:
            label_length2 = 0
            for i in range(indices.shape[0]):
                label_length2 = np.max([label_length2, np.max(indices[i][:data_num[i]])])
            label_length2 += 1
            if label_length < label_length2:
                #expaning labels and confidence arrays, as the new file appears having mode of them
                for i in range(label_length2 - label_length):
                    merged_label = np.append(merged_label, 0)
                    merged_confidence = np.append(merged_confidence, 0.0)
                label_length = label_length2
        
        for i in range(labels_seg.shape[0]):
            temp_label = np.zeros((data_num[i]),dtype=int)
            pred_confidence = confidence[i][:data_num[i]]
            temp_confidence = merged_confidence[indices[i][:data_num[i]]]

            temp_label[temp_confidence >= pred_confidence] = merged_label[indices[i][:data_num[i]]][temp_confidence >= pred_confidence]
            temp_label[pred_confidence > temp_confidence] = labels_seg[i][:data_num[i]][pred_confidence > temp_confidence]

            merged_confidence[indices[i][:data_num[i]][pred_confidence > temp_confidence]] = pred_confidence[pred_confidence > temp_confidence]
            merged_label[indices[i][:data_num[i]]] = temp_label

    if len(pred_list) > 0:
        #concatenating source points with the final labels and writing out resulting file
        final_labels = np.ndarray((merged_label.shape[0], 1), np.int64)
        final_labels[:,0] = merged_label# + 1        
        points_path = os.path.join(args.datafolder, category + LOAD_FROM_EXT)
        if LOAD_FROM_EXT == ".las":
            points, _, _ = read_xyz_label_from_las(points_path)
        else:
            print('Reading {}'.format(points_path))
            points = np.loadtxt(points_path)
        if SAVE_TO_EXT == '.las':
            save_xyz_label_to_las(output_path, points, final_labels)
        else:
            final = np.concatenate([points, final_labels], axis=-1)        
            print('Writing {}'.format(output_path))
            np.savetxt(output_path, final, fmt='%1.3f %1.3f %1.3f %i %i')
    
print('Done.')

['Tile_135000_450000' 'Tile_135000_450500' 'Tile_135000_451000'
 'Tile_135000_451500' 'Tile_135000_452000' 'Tile_135000_452500'
 'Tile_135000_453000' 'Tile_135000_453500' 'Tile_135000_454000'
 'Tile_135000_454500' 'Tile_135000_455000' 'Tile_135000_455500'
 'Tile_135000_456000' 'Tile_135500_450000' 'Tile_135500_450500'
 'Tile_135500_451000' 'Tile_135500_451500' 'Tile_135500_452000'
 'Tile_135500_452500' 'Tile_135500_453000' 'Tile_135500_453500'
 'Tile_135500_454000' 'Tile_135500_454500' 'Tile_135500_455000'
 'Tile_135500_455500' 'Tile_135500_456000' 'Tile_136000_450000'
 'Tile_136000_450500' 'Tile_136000_451000' 'Tile_136000_451500'
 'Tile_136000_452000' 'Tile_136000_452500' 'Tile_136000_453000'
 'Tile_136000_453500' 'Tile_136000_454000' 'Tile_136000_454500'
 'Tile_136000_455000' 'Tile_136000_455500' 'Tile_136000_456000'
 'Tile_136500_450000' 'Tile_136500_450500' 'Tile_136500_451000'
 'Tile_136500_451500' 'Tile_136500_452000' 'Tile_136500_452500'
 'Tile_136500_453000' 'Tile_136500_45350